# Script to get the location coordinates

## To-Do
- Destination 
1. Get the appropriate location name, port/airport.
2. Get the coordinates
- Source
1. Get the coordinates, if mode is AIR then it should be the airport of that city. If ocean, then the nearest port of that city. If road then that place coordinates

In [8]:
from geopy.geocoders import ArcGIS

import pandas as pd
# For converting country name to iso_2 codes
import country_converter as coco
# Initialising geolocation object
nom = ArcGIS()

In [13]:
location = nom.geocode('Medochemie Factory A, CY')

In [14]:
location

Location(CY, (35.05017420400003, 33.22622979000005, 0.0))

In [15]:
print(str(location.latitude) + ',' + str(location.longitude))

35.05017420400003,33.22622979000005


In [14]:
str(location.latitude) + ',' + str(location.longitude)

'36.35627096300004,127.80639593300009'

In [16]:
country = nom.reverse(str(location.latitude) + ',' + str(location.longitude))

In [17]:
country

Location(2643, (35.05017420400003, 33.22622979000005, 0.0))

In [6]:
# Read ports data
import geopandas as gpd
ports = gpd.read_file('Data/attributed_ports.geojson')



In [27]:
ports[ports['geometry'].x == 28]

,Country,Function,LOCODE,Name,NameWoDiac,Status,outflows,geometry


In [9]:
ports_df = ports[['Country','Name','geometry']].copy()
ports_df['latitude'] = ports_df.geometry.apply(lambda p: p.y)
ports_df['longitude'] = ports_df.geometry.apply(lambda p: p.x)
ports_df.drop(['geometry'], axis=1,inplace=True)
ports_df['iso_country'] = coco.convert(ports_df['Country'],to='ISO2', not_found=None)

In [10]:
ports_df.head(10)

,Country,Name,latitude,longitude,iso_country
0,United Arab Emirates,Abu Dhabi,24.466667,54.366667,AE
1,United Arab Emirates,Ar Ruways,24.116667,52.733333,AE
2,United Arab Emirates,Khor al Fakkan,25.333333,56.350000,AE
3,United Arab Emirates,Mina Khalid,25.350000,55.366667,AE
4,United Arab Emirates,Mina Khalifa/Abu Dhabi,24.833333,54.666667,AE
5,United Arab Emirates,Umm al Qaiwain,25.566667,55.550000,AE
6,Antigua and Barbuda,Saint John's,17.116667,-61.850000,AG
7,Argentina,Bahía Blanca,-38.716667,-62.283333,AR
8,Argentina,Buenos Aires,-34.583333,-58.666667,AR
9,Argentina,Mar del Plata,-38.050000,-57.533333,AR


In [36]:
# Getting airports data
raw_airport_df = pd.read_csv('Data/airports.csv')
df = raw_airport_df[['type','latitude_deg','longitude_deg','name','iso_country','municipality']]
final_airport_df = df[df['type'].str.lower().str.contains('airport')]


In [18]:
# Getting airports data
raw_airport_df = pd.read_csv('Data/airports.csv')

In [45]:
country_capital = pd.read_csv('Data/country-capital-list.csv')

In [48]:
country_capital.drop(columns=['type'],axis=1,inplace=True)

In [50]:
country_capital['iso_country'] = coco.convert(country_capital['country'],to='ISO2', not_found=None)

Abkhazia not found in regex
Akrotiri and Dhekelia not found in regex
Ascension Island not found in regex
Easter Island not found in regex
Nagorno-Karabakh Republic not found in regex
Scotland not found in regex
South Ossetia not found in regex
Transnistria not found in regex
Tristan da Cunha not found in regex
Wales not found in regex


In [3]:
# Calculating aerial distance

# Importing the great_circle module from the library
from geopy.distance import great_circle,geodesic
  
# Loading the lat-long data for Kolkata & Delhi
kolkata = (22.5726, 88.3639)
delhi = (28.7041, 77.1025)
  
# Print the distance calculated in km
print("Aerial Distance",great_circle(kolkata, delhi).km)


import searoute as sr

#Define origin and destination points:
origin = [54.366667,24.466667] 
destination = [-62.283333,-38.716667]
route = sr.searoute(origin, destination)
# show route distance with unit
print(route.properties['length'])
print("{:.1f} {}".format(route.properties['length'], route.properties['units']))
# print(route)



Aerial Distance 1317.7554645657162
16038.285321256231
16038.3 km


In [5]:
# Finding road distance


import openrouteservice
""" - setup openrouteservice client with api key, you can signup https://openrouteservice.org 
      if you don't have API key. Its totaly free😊
    - After signup, you can see your API key available under the dashboard tab.
"""
API_KEY = '5b3ce3597851110001cf6248de01212ad45649ddbb4031ce99837efa'
client = openrouteservice.Client(key=API_KEY)

#set location coordinates in longitude,latitude order
coords = ((78.4033966,17.6271992),( 77.1025,28.7041))


# res = client.directions(coords)
# res['routes'][0]['summary']['distance']/1000

help(client)

Help on Client in module openrouteservice.client object:

class Client(builtins.object)
 |  Client(key=None, base_url='https://api.openrouteservice.org', timeout=60, retry_timeout=60, requests_kwargs=None, retry_over_query_limit=True)
 |  
 |  Performs requests to the ORS API services.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, key=None, base_url='https://api.openrouteservice.org', timeout=60, retry_timeout=60, requests_kwargs=None, retry_over_query_limit=True)
 |      :param key: ORS API key.
 |      :type key: string
 |      
 |      :param base_url: The base URL for the request. Defaults to the ORS API
 |          server. Should not have a trailing slash.
 |      :type base_url: string
 |      
 |      :param timeout: Combined connect and read timeout for HTTP requests, in
 |          seconds. Specify "None" for no timeout.
 |      :type timeout: int
 |      
 |      :param retry_timeout: Timeout across multiple retriable requests, in
 |          seconds.
 |      :type r

1509.5617